# Road Traffic Vehicle Counting Exam

In this exam, you will use the YOLO (You Only Look Once) object detection model to analyze a video of road traffic. Your primary task is to count the number of vehicles exiting the road at each of the four exits (located at the top, bottom, left, and right of the frame). The video for this task is provided at `Datasets/YOLO_Exam_Video.mp4`.

## Objectives
- Load and process the video using OpenCV.
- Utilize the YOLO model to detect vehicles in each frame.
- Manipulate the video frames using OpenCV.
- Track vehicles as they move across the video frames.
- Count the number of vehicles exiting via the top street.
- Count the number of vehicles exiting via the bottom street.
- Count the number of vehicles exiting via the left street.
- Count the number of vehicles exiting via the right street.


## Setup Environment

Before you begin, ensure you have the necessary libraries installed. You will need `opencv`, and `ultralytics` among others.
If these are not installed, you should install them.

In [ ]:
!pip install opencv-python


In [ ]:
!pip install ultralytics


## Load the YOLO Model

You will first need to load the YOLO model, there is a custom model for this exam called `YOLO_Model.pt` under Datasets file ⚠️ PLEASE USE THIS MODEL AND DO NOT USE ANY OTHER MODEL ⚠️.
Write the code to load the YOLO model below:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import cv2
import numpy as np
from ultralytics import YOLO
model = YOLO('/content/drive/MyDrive/YOLO_Model.pt')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Prepare the Video Capture

Create a variable to capture the video frames, you can use `cv2.VideoCapture()` to achive this.

## Get Video Information

You can use `cv2` library to get these information fro the `VideoCapture()` variable you created to extract these information:
* `height`: Video's height.
* `width`: Video's width.
* `fps`: Video's frames.

In [ ]:
video_path = '/content/YOLO_Video.mp4'
video = cv2.VideoCapture(video_path)
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
fps = video.get(cv2.CAP_PROP_FPS)


## Prepare Video Writer to Store the Output

Create a variable that uses `cv2.VideoCapture()` to save the video with the bounding boxes, specified Region of Interest rectangle (ROI) and the counted vehicles on each sides. You will need to make the video with the same `fps`, `width`, `height`, and specify the codec and output path of the video.

In [ ]:
output_path = '/content/drive/My Drive/YOLO_Video_Output.mp4'
VIDEO_CODEC = "mp4v"
output_video = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*VIDEO_CODEC), fps, (width, height))

## Expermint to get the `(x1, y1, x2, y2)` of the ROI

### Hint
When defining the Regions of Interest (ROIs) for detecting vehicles exiting the road, consider the coordinates that outline the relevant areas. These coordinates can be specified as `(x1, y1, x2, y2)`, where `(x1, y1)` is the top-left corner and `(x2, y2)` is the bottom-right corner of the rectangle.

Experiment with different coordinates to accurately cover the streets where vehicles exit. Visualize these ROIs by drawing rectangles on a frame of the video and adjust the coordinates as needed until you capture the desired areas effectively.

In [ ]:
roi_top = (int(width * 0.55), 0, int(width * 0.61), int(height * 0.33))
roi_bottom = (int(width * 0.35), int(height * 0.77), int(width * 0.41), height)
roi_left = (int(width * 0.15), int(height * 0.35), int(width * 0.3), int(height * 0.5))
roi_right = (int(width * 0.62), int(height * 0.53), int(width * 0.8), int(height * 0.66))

## Process Video Frames and Identify Vehicles Left the Road Form Each Street

For each frame in the video, use the YOLO model to detect and track vehicles. You will need to write a loop that processes each frame and applies the YOLO model.

In each frame, after detecting the vehicles, determine whether they exited the road via the top, bottom, left, or right street. You can use the positions of the bounding boxes provided by YOLO to do this.

- The video should display bounding boxes around the detected objects.
- The video should display your name on the top.
- It should display the calculated center of each vehicle.
- The video should display the confidence score, alongside the object ID and class ID of each detected and tracked object.
- The video should display the rectangles representing the regions where you count the vehicles that have crossed and exited.
- The video should display the number of vehicles that exited via the top street.
- The video should display the number of vehicles that exited via the bottom street.
- The video should display the number of vehicles that exited via the left street.
- The video should display the number of vehicles that exited via the right street.

In [ ]:

zones = {
    'upper': roi_top,
    'lower': roi_bottom,
    'left_side': roi_left,
    'right_side': roi_right
}

object_count = {'upper': 0, 'lower': 0, 'left_side': 0, 'right_side': 0}
tracked_objects = {'upper': set(), 'lower': set(), 'left_side': set(), 'right_side': set()}

while video.isOpened():
    ret, frame = video.read()
    if not ret:
        break

    results = model(frame)

    for zone_name, (x1, y1, x2, y2) in zones.items():
        cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)

    for result in results:
        boxes = result.boxes
        for box in boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            class_id = int(box.cls)
            confidence = box.conf.item()

            center_x = int((x1 + x2) / 2)
            center_y = int((y1 + y2) / 2)

            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.circle(frame, (center_x, center_y), 5, (0, 0, 255), -1)
            cv2.putText(frame, f'ID: {class_id} Conf: {confidence:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            print(f'Object ID: {box.id}, Center: ({center_x}, {center_y})')

            if not hasattr(box, 'id') or box.id is None:
                continue

            for zone_name, (zx1, zy1, zx2, zy2) in zones.items():
                if zx1 < center_x < zx2 and zy1 < center_y < zy2:
                    if box.id not in tracked_objects[zone_name]:
                        tracked_objects[zone_name].add(box.id)
                        object_count[zone_name] += 1
                        print(f'Counted in {zone_name}: {object_count[zone_name]}')

    cv2.putText(frame, f'Top: {object_count["upper"]}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    cv2.putText(frame, f'Bottom: {object_count["lower"]}', (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    cv2.putText(frame, f'Left: {object_count["left_side"]}', (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    cv2.putText(frame, f'Right: {object_count["right_side"]}', (50, 200), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    cv2.putText(frame, '', (width - 200, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

    output_video.write(frame)

video.release()
output_video.release()


## Save and Submit Your Work

https://drive.google.com/file/d/1wcbaisgMvqbKmZZbdNWgKPsthscw3S3y/view?usp=sharing

Submit both the notebook and the output video